# 下载modelscope模型

In [16]:
# import torch
# from modelscope import Model, AutoTokenizer


# model = Model.from_pretrained("modelscope/Llama-2-7b-ms", revision='v1.0.1', device_map='auto', torch_dtype=torch.float16)
# tokenizer = AutoTokenizer.from_pretrained("modelscope/Llama-2-7b-ms", revision='v1.0.1')

# prompt = "Hey, are you conscious? Can you talk to me?"
# inputs = tokenizer(prompt, return_tensors="pt")

# # Generate
# generate_ids = model.generate(inputs.input_ids.to(model.device), max_length=30)
# print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

# 导包与传入参数

In [1]:
import os
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, DataCollatorForSeq2Seq
login_token = os.getenv('HF_TOKEN')

C:\Language\Anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Language\Anaconda3\envs\llm\lib\site-packages\bitsandbytes\cuda_setup\main.py:112: UserWarning: Loading CUDA version: BNB_CUDA_VERSION=18


  warn((f'\n\n{"="*80}\n'


bin C:\Language\Anaconda3\envs\llm\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
import peft
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

**TODO:内置参数**（不知道是啥）

In [3]:
device_map = "auto"
# world_size = int(os.environ.get("WORLD_SIZE", 1))
# if world_size != 1:
#     device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
#     gradient_accumulation_steps = gradient_accumulation_steps // world_size

# # Check if parameter passed or if set within environ
# use_wandb = len(wandb_project) > 0 or (
#     "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
# )
# # Only overwrite environ if wandb param passed
# if len(wandb_project) > 0:
#     os.environ["WANDB_PROJECT"] = wandb_project
# if len(wandb_watch) > 0:
#     os.environ["WANDB_WATCH"] = wandb_watch
# if len(wandb_log_model) > 0:
#     os.environ["WANDB_LOG_MODEL"] = wandb_log_model

# 数据加载

In [4]:
from datasets import load_dataset
data_path = './train_v3.json'

if data_path.endswith(".json") or data_path.endswith(".jsonl"):
    data = load_dataset("json", data_files=data_path)
else:
    data = load_dataset(data_path)

Using custom data configuration default-265d39595e7fd67a
Found cached dataset json (F:/huggingface/datasets/json/default-265d39595e7fd67a/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.34it/s]


**查看原始数据**

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 55
    })
})

In [8]:
for index, item in enumerate(data['train']):
    if index < 5:
        print(item)
    else:
        break

{'instruction': 'Please extract the entity of person in the input sentence given below , the entity of person refers to the entity that represents the identity or role of a specific person in the input sentense .', 'input': 'At the moment Shalev is in Levico Terme , Trento , Italy for a conference .', 'output': '<im_start> I can extract entities for you, the extracted entities are <<< Shalev >>>  <im_end>'}
{'instruction': 'Please extract the entity of person in the input sentence given below , the entity of person refers to the entity that represents the identity or role of a specific person in the input sentense .', 'input': 'If you already own a Beavertail cactus , you can make your own cuttings .', 'output': '<im_start> I can extract entities for you, the extracted entities are <<< you >>>  <im_end>'}
{'instruction': 'Please extract the entity of person in the input sentence given below , the entity of person refers to the entity that represents the identity or role of a specific p

# 预训练模型加载

In [9]:
# 传入参数
base_model = 'meta-llama/Llama-2-7b-hf'
# base_model = r'C:\Users\joyho\.cache\modelscope\hub\modelscope\Llama-2-7b-ms'
output_dir = 'lora-alpaca_origin'
batch_size = 128
micro_batch_size = 4
lora_r = 32 # lora的维度 矩阵分解
lora_alpha = 16 # lora的权重
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "v_proj",
] # 指定在哪些线形层做lora

## LLM加载

In [10]:
# 加载base model
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map=device_map,
    token=login_token
)

Loading checkpoint shards: 100%|███████████████████████████████████████████| 2/2 [00:02<00:00,  1.19s/it]
C:\Language\Anaconda3\envs\llm\lib\site-packages\transformers\utils\hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [11]:
# Opts: 量化训练，减小显存使用，训练效果下降
model = prepare_model_for_int8_training(model)

C:\Language\Anaconda3\envs\llm\lib\site-packages\peft\utils\other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


**查看模型结构**

In [12]:
model.modules

<bound method Module.modules of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (

## LoRA训练配置

In [13]:
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# 数据预处理

## 预训练分词器加载

In [15]:
# 加载分词器
tokenizer = LlamaTokenizer.from_pretrained(base_model)

**配置分词器**

In [16]:
# 设置pad token为unk
tokenizer.pad_token_id = 0
# 设置在字符串左边填充
tokenizer.padding_side = "left"

pad token: 一种特殊的令牌，用于使令牌阵列具有相同的大小以进行批处理。然后将被注意力机制或损失计算忽略。

* \<UNK\>: 低频词或未在词表中的词
* \<PAD\>: 补全字符
* \<GO\>/\<SOS\>: 句子起始标识符
* \<EOS\>: 句子结束标识符
* \[SEP\]：两个句子之间的分隔符
* \[MASK\]：填充被掩盖掉的字符

Bert标识符
* \[CLS\]：句子开头，以\[CLS\]开头，随后跟句子的第一个单词

**调用分词器进行分词**

In [17]:
def tokenize(prompt, add_eos_token=True):
    '''对输入str进行分词
    
    :param prompt 根据模板构造好的提示词（字符串）
    :param add_eos_token 如果分词结果最后一位不是<句子起始标识符>，手动添加。
    
    输出：字典dict_keys(['input_ids', 'attention_mask','labels'])，其中labels就是input_ids。
    '''
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    ) # dict_keys(['input_ids', 'attention_mask'])
    
    # 如果分词结果最后一位不是<句子起始标识符> and 未被cutoff and 需要添加eos，手动添加
    # TODO: 被cutoff的句子最后一位就不是<EOS>，不处理吗？
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    # 额外添加label key，应该是跟训练有关系
    result["labels"] = result["input_ids"].copy()
    return result

## prompt构造

In [18]:
from utils.prompter import Prompter
prompt_template_name = "alpaca" # TODO: 这模板也不知道是啥文件
cutoff_len = 1024 # 超出长度的prompt会被切断。TODO: 这个没用啊
prompter = Prompter(prompt_template_name)
train_on_inputs = True # 是否不计算输入的损失
add_eos_token = False # 是否添加句子结束标识符

**查看构造&分词结果**

In [20]:
data_index = 0
full_prompt = prompter.generate_prompt(
                data['train'][data_index]["instruction"],
                data['train'][data_index]["input"],
                data['train'][data_index]["output"],
            )
tokenized_full_prompt = tokenizer(
                        full_prompt,
                        truncation=True,
                        max_length=cutoff_len,
                        padding=False,
                        return_tensors=None,
                        )
print('='*100)
print(f"length of seq: {len(tokenized_full_prompt['input_ids'])}")
print('='*100)
print(full_prompt) # 查看构造后的结果
print('='*100)
print(tokenized_full_prompt) # 查看分词后的结果
print('='*100)


length of seq: 139
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please extract the entity of person in the input sentence given below , the entity of person refers to the entity that represents the identity or role of a specific person in the input sentense .

### Input:
At the moment Shalev is in Levico Terme , Trento , Italy for a conference .

### Response:
<im_start> I can extract entities for you, the extracted entities are <<< Shalev >>>  <im_end>
{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 12148, 6597, 278, 7855, 310, 2022, 297, 278, 1881, 10541, 2183, 2400, 1919, 278, 7855, 310, 2022, 14637, 304, 278, 7855, 393, 11524, 278, 10110, 470, 6297, 310, 263, 2702, 

In [21]:
def generate_and_tokenize_prompt(data_point):
    # 根据所有features和labels进行full_prompt构造，并作分词
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],)
    tokenized_full_prompt = tokenize(full_prompt)
    
    # 如果mask input loss
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    # TODO: 额外设置了俩key，不知道有啥用
    tokenized_full_prompt["pos"] = [1111,1111]
    tokenized_full_prompt["neg"] = [0000,0000]

    return tokenized_full_prompt

## 分割数据集&乱序

In [22]:
val_set_size = 10
if val_set_size > 0:
    train_val = data["train"].train_test_split(
        test_size=val_set_size, shuffle=True, seed=42
    )

    train_data = (
        train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    )
    val_data = (
        train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    )
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

100%|███████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 338.59ex/s]


In [23]:
len(train_data[0]['input_ids'])

154

## Data Collector

In [24]:
class CustomDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __call__(self, features, return_tensors=None):
        feat = super().__call__(features, return_tensors=None)
#        print("feat:")
#        print(feat)
        input_ids_batch = feat["input_ids"].tolist()
        origin_arr = []
        pos_arr = []
        neg_arr = []
        for input_ids in input_ids_batch:
            input_str = "_".join([str(item) for item in input_ids])
            origin_str = "_".join(origin)
            prefix_str = "_".join(prefix)
            suffix_str = "_".join(suffix)
            sentence_str = "_".join(input)

            origin_start = input_str.find(origin_str)

            origin_index = input_str[0:origin_start + len(origin_str)].count("_") + 1

            origin_arr.append(origin_index)

            prefix_start = input_str.find(prefix_str)
            prefix_index = input_str[0:prefix_start + len(prefix_str)].count("_") + 1

            input_start = input_str.find(sentence_str)
            input_index = input_str[0:input_start + len(sentence_str)].count("_") + 1


            suffix_start = input_str.find(suffix_str)
            suffix_index = input_str[0:suffix_start].count("_") - 1

            entity_ids = input_ids[prefix_index: suffix_index + 1]
            overlap_ids = longest_common_sublist(input_ids[input_index:prefix_index], entity_ids)

            sentence_list = input_ids[input_index:prefix_index]
            
            pos_start = input_index + overlap_ids[0] - overlap_ids[2]
            pos_end = input_index + overlap_ids[0] - overlap_ids[2] + len(entity_ids)
            pos_arr.append([pos_start, pos_end - 1])
            neg_arr.append([pos_start - 1, pos_start - 2, pos_end, pos_end + 1])
            
        feat["origin"] = torch.tensor(origin_arr)
        feat["pos"] = torch.tensor(pos_arr)
        feat["neg"] = torch.tensor(neg_arr)

        
        return feat

# 训练

## 训练配置参数

In [25]:
batch_size = 1
micro_batch_size = 1
num_epochs = 5
learning_rate = 3e-4
cutoff_len = 256
resume_from_checkpoint = False

# TODO: 下面都是啥玩意
gradient_accumulation_steps = batch_size // micro_batch_size
group_by_length = False
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp != 1:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    gradient_accumulation_steps = gradient_accumulation_steps // world_size

# TODO: wandb是啥玩意
wandb_project = ''
wandb_run_name = ""
wandb_watch = ""  # options: false | gradients | all
wandb_log_model = ""
# Check if parameter passed or if set within environ
use_wandb = len(wandb_project) > 0 or (
    "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
)
# Only overwrite environ if wandb param passed
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
if len(wandb_watch) > 0:
    os.environ["WANDB_WATCH"] = wandb_watch
if len(wandb_log_model) > 0:
    os.environ["WANDB_LOG_MODEL"] = wandb_log_model

**如果断点续训**

In [26]:
if resume_from_checkpoint:
    # Check the available weights and load them
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )  # Full checkpoint
    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )  # only LoRA model - LoRA config above has to fit
        resume_from_checkpoint = (
            False  # So the trainer won't try loading its state
        )
    # The two files above have a different name depending on how they were saved, but are actually the same.
    if os.path.exists(checkpoint_name):
        print(f"Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {checkpoint_name} not found")


**查看可训练参数个数**

In [27]:
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.24836028248556738


## Train Class

In [28]:
class ContrastiveTrainer(Trainer):
    def contrastive_loss(self, hidden_states, origin, pos, neg):
        # hidden_states:  B * L * D
        # origin:  B
        # pos : B * P
        # neg : B * N
        # B * 1 * D      pos_e:  B * P * D  neg_e: B * N * D
        # P = 2  N = 4

        origin_e = torch.gather(hidden_states, 1, origin.unsqueeze(-1).unsqueeze(-1).expand(-1,-1,hidden_states.shape[-1]))
        pos_e = torch.gather(hidden_states, 1, pos.unsqueeze(-1).expand(-1,-1,hidden_states.shape[-1]))
        neg_e = torch.gather(hidden_states, 1, neg.unsqueeze(-1).expand(-1,-1,hidden_states.shape[-1]))

        origin_e = origin_e / origin_e.norm(dim=2, keepdim=True)
        pos_e = pos_e / pos_e.norm(dim=2, keepdim=True)
        neg_e = neg_e / neg_e.norm(dim=2, keepdim=True)


        pos_score = (torch.mul(origin_e, pos_e).sum(dim=2)).sum(dim=1)
        neg_score = (torch.mul(origin_e, neg_e).sum(dim=2)).sum(dim=1)

        cl_loss = -torch.log(1e-10 + torch.sigmoid(pos_score - neg_score)).mean()

        print("cl_loss:", cl_loss)
        return cl_loss


    def compute_loss(self, model, inputs, return_outputs=False):

        origin = copy.deepcopy(inputs["origin"])
        pos = copy.deepcopy(inputs["pos"])
        neg = copy.deepcopy(inputs["neg"])

        if "origin" in inputs:
            inputs.pop("origin")
        if "pos" in inputs:
            inputs.pop("pos")
        if "neg" in inputs:
            inputs.pop("neg")
        
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs, output_hidden_states=True)

        cts_loss = self.contrastive_loss(outputs["hidden_states"][26], origin, pos, neg)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            if is_peft_available() and isinstance(model, PeftModel):
                model_name = unwrap_model(model.base_model)._get_name()
            else:
                model_name = unwrap_model(model)._get_name()
            if model_name in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        
        loss += 0.001 * cts_loss
#        print(0.0015)
        return (loss, outputs) if return_outputs else loss

In [29]:
trainer = ContrastiveTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=micro_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps" if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=200 if val_set_size > 0 else None,
        save_steps=200,
        output_dir=output_dir,
        save_total_limit=3,
        load_best_model_at_end=True if val_set_size > 0 else False,
        ddp_find_unused_parameters=False if ddp else None,
        group_by_length=group_by_length,
        report_to="wandb" if use_wandb else None,
        run_name=wandb_run_name if use_wandb else None,
    ),
    data_collator=CustomDataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
model.config.use_cache = False